In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

from astropy.io import fits
from astropy.table import Table
from astropy.coordinates import SkyCoord

from easycat.lightcurve.reprocess import ReprocessFactory
from easycat.lightcurve.reprocess import WiseReprocessor

## Downloading WISE Lightcurve
Todoing... ...

## Reprocessing WISE Data

In [ ]:
sdss_pos = SkyCoord(ra=153.38352, dec=26.47322, unit="deg", frame="fk5")

with fits.open("data/9626-57875-0685.fits") as hdul:
    lcurve = Table(hdul[1].data).to_pandas()


print(type(lcurve))
print(lcurve.columns)

In [ ]:
repro = ReprocessFactory.get(metadata={
    "telescope": "WISE"
})

lcurve_repro = repro.reprocess(lcurve=lcurve, pos_ref=sdss_pos)

In [ ]:
fig, axs = plt.subplots(1, 2, sharey=True, sharex=True, figsize=(6, 3))

axs[0].scatter(lcurve.mjd, lcurve.w1mag, c="tab:blue", s=1)
axs[0].scatter(lcurve.mjd, lcurve.w2mag, c="tab:red",  s=1)
axs[0].yaxis.set_inverted(True)

axs[1].scatter(lcurve_repro.mjd, lcurve_repro.w1mag, c="tab:blue", s=1)
axs[1].scatter(lcurve_repro.mjd, lcurve_repro.w2mag, c="tab:red",  s=1)
axs[1].set_yticks([])

fig.tight_layout(w_pad=0)

plt.show()

## Generating Long-term WISE Lightcurve

Then let us generate long-term WISE lightcurve,

In [ ]:
longterm = repro.generate_longterm_lcurve(lcurve_repro, max_interval=50)

dpi = 200
fs = (800/dpi, 400/dpi)

fig = plt.figure(figsize=fs, dpi=dpi)
gs = GridSpec(2, 1, figure=fig, wspace=0, hspace=0, height_ratios=(2, 1))

ax_mag = fig.add_subplot(gs[0])
ax_color = fig.add_subplot(gs[1])
ax_color.sharex(ax_mag)
ax_mag.set_xticks([])

mjd_start = longterm.mjd[0]
rmjd = longterm.mjd - mjd_start

param_errorbar = {
    "linestyle": "none",
    "capsize": 2,
    "marker": ".",
    "markersize": 5
}

w1data = (rmjd, longterm.w1mag,     longterm.w1sigmag)
w2data = (rmjd, longterm.w2mag+1, longterm.w2sigmag)
ax_mag.invert_yaxis()

ax_mag.errorbar(*w1data, color="tab:blue", **param_errorbar)
ax_mag.errorbar(*w2data, color="tab:red",  **param_errorbar)

color = longterm.w1mag - longterm.w2mag
sigcolor = np.sqrt(longterm.w1sigmag**2 + longterm.w2sigmag**2)
ax_color.errorbar(rmjd, color, sigcolor, color="green", **param_errorbar)

plt.show()